In [13]:
import hashlib
import time

class Block:
    def __init__(self, data, previous_hash=None):
        self.timestamp = time.time()
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.compute_hash()

    def compute_hash(self):
        block_header = str(self.timestamp) + str(self.data) + str(self.previous_hash) + str(self.nonce)
        return hashlib.sha256(block_header.encode()).hexdigest()

    def mine_block(self, difficulty):
        prefix = '0' * difficulty
        while self.hash[:difficulty] != prefix:
            self.nonce += 1
            self.hash = self.compute_hash()
        print("Block mined: ", self.hash)


class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.difficulty = 5

    def create_genesis_block(self):
        return Block("Genesis Block")

    def add_block(self, new_block):
        new_block.previous_hash = self.chain[-1].hash
        new_block.mine_block(self.difficulty)
        self.chain.append(new_block)

    def print_chain(self):
        for block in self.chain:
            print("Block:", block.hash)
            print("Previous Hash:", block.previous_hash)
            print("Data:", block.data)
            print("Timestamp:", block.timestamp)
            print()

blockchain = Blockchain()
block_A = Block("Block A")
block_B = Block("Block B")
block_C = Block("Block C")

blockchain.add_block(block_A)
blockchain.add_block(block_B)
blockchain.add_block(block_C)

blockchain.print_chain()

Block mined:  00000d4474b61214e0a13191237f7975ff8b3c7b7af21b5425086d1fa5720541
Block mined:  00000005f93098f88a08dcc904930442056d6db75563b974c94c6e0f047bdf24
Block mined:  000001d1ef538452b886322737f4fe935a0f0d2c244f27258b8e66cf4306b59b
Block: a970a9f1a5dc17fa962fdbf467098476b4fb042c5a1fdba0780225a1f5b0a700
Previous Hash: None
Data: Genesis Block
Timestamp: 1711983070.6137257

Block: 00000d4474b61214e0a13191237f7975ff8b3c7b7af21b5425086d1fa5720541
Previous Hash: a970a9f1a5dc17fa962fdbf467098476b4fb042c5a1fdba0780225a1f5b0a700
Data: Block A
Timestamp: 1711983070.6137257

Block: 00000005f93098f88a08dcc904930442056d6db75563b974c94c6e0f047bdf24
Previous Hash: 00000d4474b61214e0a13191237f7975ff8b3c7b7af21b5425086d1fa5720541
Data: Block B
Timestamp: 1711983070.6137257

Block: 000001d1ef538452b886322737f4fe935a0f0d2c244f27258b8e66cf4306b59b
Previous Hash: 00000005f93098f88a08dcc904930442056d6db75563b974c94c6e0f047bdf24
Data: Block C
Timestamp: 1711983070.6137257



In [14]:
from pymerkle import *

In [68]:
import hashlib

class MerkleTree:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.hash = None

    def compute_hash(self):
        data_hash = hashlib.sha256(self.data.encode()).hexdigest().encode()
        self.hash = data_hash
        return data_hash

    def insert(self, data_list):
        if len(data_list) == 0:
            return None

        mid = len(data_list) // 2
        self.data = data_list[mid]

        if len(data_list) > 1:
            self.left = MerkleTree(None)
            self.left.insert(data_list[:mid])
            self.right = MerkleTree(None)
            self.right.insert(data_list[mid + 1:])

        self.compute_hash()

    def print_tree(self, level=0, branch=None):
        if self.data is None:
            return

        prefix = " " * level * 4
        if branch == "left":
            prefix += "L-> "
        elif branch == "right":
            prefix += "R-> "

        print(prefix + f"- {self.data}")
        if self.left:
            self.left.print_tree(level + 1, "left")
        if self.right:
            self.right.print_tree(level + 1, "right")

    def search(self, target):
        if self.data == target:
            return True
        if self.left and target < self.data:
            return self.left.search(target)
        elif self.right and target > self.data:
            return self.right.search(target)
        return False

    def get_proof_of_inclusion(self, target):
        proof = []
        node = self
        while node.data != target:
            if target < node.data and node.left:
                proof.append((node.data, node.right.hash))
                node = node.left
            elif target > node.data and node.right:
                proof.append((node.data, node.left.hash))
                node = node.right
            else:
                raise ValueError("Target not found in the Merkle tree")
        return proof

data_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
merkle_tree = MerkleTree(None)
merkle_tree.insert(data_list)

print("Merkle Tree:")
merkle_tree.print_tree()

target_data = 'A'
proof = merkle_tree.get_proof_of_inclusion(target_data)
print("\nProof of Inclusion for", target_data, ":", proof)

Merkle Tree:
- F
    L-> - C
        L-> - B
            L-> - A
        R-> - E
            L-> - D
    R-> - I
        L-> - H
            L-> - G
        R-> - J

Proof of Inclusion for A : [('F', b'a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919271f9fa140b0508c6c'), ('C', b'a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0e22207b8981233ec58'), ('B', None)]


In [69]:
with open('current_state', 'w') as f:
    f.write(tree.__repr__())